# Phase 4: Peri-Urban Growth Classification

## Objective
Identify districts undergoing peri-urban transformation using explainable,
rule-based logic derived from address update behavior and youth population signals.

## Input
- `data/processed/district_features.csv` (district × month)

## Output
- `data/final/peri_urban_labels.csv`

## Classification Rules
| Label | Conditions |
|-------|------------|
| Peri-Urban Growth | velocity > 0.12 AND youth_ratio > 0.38 AND growth_3m_consistent |
| Declining | velocity < -0.05 |
| Stable Urban | youth_ratio >= 0.40 |
| Rural Stable | All other cases |

## Growth Confidence Formula
```
growth_confidence = 0.5 * norm(velocity) + 0.3 * youth_ratio + 0.2 * growth_flag
```

## Cell 1: Import Libraries and Set Paths

In [1]:
"""
Import Required Libraries
-------------------------
- pandas: Data manipulation
- pathlib: Cross-platform file path handling
"""
import pandas as pd
from pathlib import Path

# Define project paths
PROJECT_ROOT = Path.cwd().parent
INPUT_PATH = PROJECT_ROOT / "data" / "processed" / "district_features.csv"
OUTPUT_PATH = PROJECT_ROOT / "data" / "final" / "peri_urban_labels.csv"

# Ensure output directory exists
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

print(f"✅ Libraries imported")
print(f"📂 Input: {INPUT_PATH}")
print(f"📂 Output: {OUTPUT_PATH}")

✅ Libraries imported
📂 Input: d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\processed\district_features.csv
📂 Output: d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\final\peri_urban_labels.csv


## Cell 2: Load Feature Data from Phase 2

In [2]:
"""
Load Feature Data
-----------------
Read the district-month feature table from Phase 2.
We need: address_velocity, youth_ratio, growth_3m_consistent
"""
df = pd.read_csv(INPUT_PATH)

# Convert growth_3m_consistent to boolean if stored as string
if df['growth_3m_consistent'].dtype == 'object':
    df['growth_3m_consistent'] = df['growth_3m_consistent'].map({'True': True, 'False': False})

print(f"✅ Loaded {len(df):,} rows from Phase 2 output")
print(f"\n📊 Columns: {list(df.columns)}")
print(f"\n📋 Key feature statistics:")
print(f"   address_velocity: min={df['address_velocity'].min():.4f}, max={df['address_velocity'].max():.4f}")
print(f"   youth_ratio: min={df['youth_ratio'].min():.4f}, max={df['youth_ratio'].max():.4f}")
print(f"   growth_3m_consistent: {df['growth_3m_consistent'].sum()} True values")
print(f"\n📋 Sample rows:")
df[['district_id', 'month', 'address_velocity', 'youth_ratio', 'growth_3m_consistent']].head(10)

✅ Loaded 240 rows from Phase 2 output

📊 Columns: ['district_id', 'month', 'address_velocity', 'migration_index', 'youth_ratio', 'digital_gap', 'growth_3m_consistent']

📋 Key feature statistics:
   address_velocity: min=-0.5745, max=1.7011
   youth_ratio: min=0.3859, max=0.4126
   growth_3m_consistent: 5 True values

📋 Sample rows:


,district_id,month,address_velocity,youth_ratio,growth_3m_consistent
0,GJ_AMD,2024-01,0.0000,0.4038,False
1,GJ_AMD,2024-02,0.5993,0.4038,False
2,GJ_AMD,2024-03,-0.1842,0.4038,False
3,GJ_AMD,2024-04,-0.1129,0.4038,False
4,GJ_AMD,2024-05,0.8165,0.4038,False
5,GJ_AMD,2024-06,-0.1249,0.4038,False
6,GJ_AMD,2024-07,-0.3278,0.4038,False
7,GJ_AMD,2024-08,0.3106,0.4038,False
8,GJ_AMD,2024-09,0.1677,0.4038,False
9,GJ_AMD,2024-10,-0.4915,0.4038,False


## Cell 3: Apply Peri-Urban Classification Rules

**Classification Logic (applied in order):**

1. **Peri-Urban Growth**: Districts showing sustained growth pattern
   - address_velocity > 0.12 (high growth rate)
   - youth_ratio > 0.38 (significant youth population)
   - growth_3m_consistent = TRUE (sustained 3-month growth)

2. **Declining**: Districts losing population
   - address_velocity < -0.05 (negative growth)

3. **Stable Urban**: Mature urban districts
   - youth_ratio >= 0.40 (high youth concentration)

4. **Rural Stable**: Default for all other districts

In [3]:
"""
Apply Peri-Urban Classification Rules
--------------------------------------
Each district-month is classified based on explicit, explainable rules.

Rule Priority (applied in order):
1. Peri-Urban Growth: High velocity + youth + sustained growth
2. Declining: Negative velocity indicates outflow
3. Stable Urban: High youth ratio indicates mature urban area
4. Rural Stable: Default category for remaining districts
"""

def classify_peri_urban(row) -> str:
    """
    Classify a district-month into one of 4 peri-urban categories.
    
    Args:
        row: DataFrame row with address_velocity, youth_ratio, growth_3m_consistent
    
    Returns:
        One of: 'Peri-Urban Growth', 'Declining', 'Stable Urban', 'Rural Stable'
    """
    velocity = row['address_velocity']
    youth = row['youth_ratio']
    growth_consistent = row['growth_3m_consistent']
    
    # Rule 1: Peri-Urban Growth
    # Districts with high address velocity, significant youth population,
    # AND sustained 3-month growth are classified as peri-urban growth zones.
    # These are areas transitioning from rural to urban.
    if velocity > 0.12 and youth > 0.38 and growth_consistent:
        return "Peri-Urban Growth"
    
    # Rule 2: Declining
    # Districts with negative address velocity are experiencing outflow.
    # This indicates population decline or reverse migration.
    elif velocity < -0.05:
        return "Declining"
    
    # Rule 3: Stable Urban
    # Districts with high youth ratio (>=40%) are mature urban centers.
    # Youth tend to concentrate in cities for jobs and education.
    elif youth >= 0.40:
        return "Stable Urban"
    
    # Rule 4: Rural Stable (default)
    # All remaining districts are classified as stable rural areas.
    # These have moderate velocity and lower youth concentration.
    else:
        return "Rural Stable"

# Apply classification to each row
df['peri_urban_label'] = df.apply(classify_peri_urban, axis=1)

print(f"✅ Peri-urban labels assigned")
print(f"\n📊 Label distribution:")
print(df['peri_urban_label'].value_counts().sort_index())
print(f"\n📋 Sample classified rows:")
df[['district_id', 'month', 'address_velocity', 'youth_ratio', 'growth_3m_consistent', 'peri_urban_label']].head(12)

✅ Peri-urban labels assigned

📊 Label distribution:
peri_urban_label
Declining            92
Peri-Urban Growth     1
Rural Stable         90
Stable Urban         57
Name: count, dtype: int64

📋 Sample classified rows:


,district_id,month,address_velocity,youth_ratio,growth_3m_consistent,peri_urban_label
0,GJ_AMD,2024-01,0.0000,0.4038,False,Stable Urban
1,GJ_AMD,2024-02,0.5993,0.4038,False,Stable Urban
2,GJ_AMD,2024-03,-0.1842,0.4038,False,Declining
3,GJ_AMD,2024-04,-0.1129,0.4038,False,Declining
4,GJ_AMD,2024-05,0.8165,0.4038,False,Stable Urban
5,GJ_AMD,2024-06,-0.1249,0.4038,False,Declining
6,GJ_AMD,2024-07,-0.3278,0.4038,False,Declining
7,GJ_AMD,2024-08,0.3106,0.4038,False,Stable Urban
8,GJ_AMD,2024-09,0.1677,0.4038,False,Stable Urban
9,GJ_AMD,2024-10,-0.4915,0.4038,False,Declining


## Cell 4: Compute Growth Confidence Score

**Formula:**
```
growth_confidence = 0.5 * normalized(address_velocity)
                  + 0.3 * youth_ratio
                  + 0.2 * (1 if growth_3m_consistent else 0)
```

**Normalization:**
- address_velocity is min-max normalized **per month**
- youth_ratio is already between 0 and 1
- growth_3m_consistent is converted to 0 or 1

In [4]:
"""
Compute Growth Confidence Score
--------------------------------
A normalized score between 0 and 1 that indicates how confident we are
that a district is experiencing growth transformation.

Components:
- 50% weight: Normalized address velocity (growth rate)
- 30% weight: Youth ratio (demographic signal)
- 20% weight: Sustained growth flag (consistency)

Normalization:
- address_velocity is normalized per month using min-max scaling
- This ensures fair comparison across months with different baselines
"""

def normalize_velocity_per_month(group):
    """
    Min-max normalize address_velocity within each month.
    
    This ensures that velocity is comparable across months,
    since different months may have different baseline activity levels.
    
    Formula: (x - min) / (max - min)
    """
    velocity = group['address_velocity']
    v_min = velocity.min()
    v_max = velocity.max()
    
    # Handle edge case: if all values are the same, return 0.5
    if v_max == v_min:
        return pd.Series([0.5] * len(velocity), index=velocity.index)
    
    return (velocity - v_min) / (v_max - v_min)

# Apply month-wise normalization
df['velocity_normalized'] = df.groupby('month', group_keys=False).apply(
    normalize_velocity_per_month
)

# Convert growth_3m_consistent to numeric (0 or 1)
df['growth_flag'] = df['growth_3m_consistent'].astype(int)

# Compute growth confidence score
# Weights: velocity=0.5, youth=0.3, growth_flag=0.2
df['growth_confidence'] = (
    0.5 * df['velocity_normalized'] +
    0.3 * df['youth_ratio'] +
    0.2 * df['growth_flag']
)

# Ensure growth_confidence is between 0 and 1
df['growth_confidence'] = df['growth_confidence'].clip(0, 1)

# Round to 4 decimal places
df['growth_confidence'] = df['growth_confidence'].round(4)

print(f"✅ Growth confidence scores computed")
print(f"\n📊 Score statistics:")
print(f"   Min: {df['growth_confidence'].min():.4f}")
print(f"   Max: {df['growth_confidence'].max():.4f}")
print(f"   Mean: {df['growth_confidence'].mean():.4f}")
print(f"\n📊 Mean confidence by label:")
print(df.groupby('peri_urban_label')['growth_confidence'].mean().round(4))
print(f"\n📋 Sample rows with confidence:")
df[['district_id', 'month', 'velocity_normalized', 'youth_ratio', 'growth_flag', 'growth_confidence', 'peri_urban_label']].head(12)

✅ Growth confidence scores computed

📊 Score statistics:
   Min: 0.1193
   Max: 0.6621
   Mean: 0.3043

📊 Mean confidence by label:
peri_urban_label
Declining            0.1925
Peri-Urban Growth    0.6621
Rural Stable         0.3664
Stable Urban         0.3802
Name: growth_confidence, dtype: float64

📋 Sample rows with confidence:


,district_id,month,velocity_normalized,youth_ratio,growth_flag,growth_confidence,peri_urban_label
0,GJ_AMD,2024-01,0.500000,0.4038,0,0.3711,Stable Urban
1,GJ_AMD,2024-02,0.733350,0.4038,0,0.4878,Stable Urban
2,GJ_AMD,2024-03,0.209085,0.4038,0,0.2257,Declining
3,GJ_AMD,2024-04,0.151980,0.4038,0,0.1971,Declining
4,GJ_AMD,2024-05,1.000000,0.4038,0,0.6211,Stable Urban
5,GJ_AMD,2024-06,0.205286,0.4038,0,0.2238,Declining
6,GJ_AMD,2024-07,0.085248,0.4038,0,0.1638,Declining
7,GJ_AMD,2024-08,0.432467,0.4038,0,0.3374,Stable Urban
8,GJ_AMD,2024-09,0.361984,0.4038,0,0.3021,Stable Urban
9,GJ_AMD,2024-10,0.042137,0.4038,0,0.1422,Declining


## Cell 5: Prepare Final Output Schema

In [5]:
"""
Prepare Final Output
--------------------
Select only the required columns in the exact schema order.

Output Schema:
- district_id (string)
- month (string, YYYY-MM)
- peri_urban_label (string)
- growth_confidence (float, 0-1)
"""

# Define exact output schema
OUTPUT_COLUMNS = [
    'district_id',
    'month',
    'peri_urban_label',
    'growth_confidence'
]

# Select only required columns
df_output = df[OUTPUT_COLUMNS].copy()

# Sort by district_id and month for deterministic output
df_output = df_output.sort_values(['district_id', 'month']).reset_index(drop=True)

print(f"✅ Final output prepared: {len(df_output):,} rows")
print(f"\n📊 Columns: {list(df_output.columns)}")
print(f"\n📋 Data types:")
print(df_output.dtypes)
print(f"\n📋 First 12 rows:")
df_output.head(12)

✅ Final output prepared: 240 rows

📊 Columns: ['district_id', 'month', 'peri_urban_label', 'growth_confidence']

📋 Data types:
district_id           object
month                 object
peri_urban_label      object
growth_confidence    float64
dtype: object

📋 First 12 rows:


,district_id,month,peri_urban_label,growth_confidence
0,GJ_AMD,2024-01,Stable Urban,0.3711
1,GJ_AMD,2024-02,Stable Urban,0.4878
2,GJ_AMD,2024-03,Declining,0.2257
3,GJ_AMD,2024-04,Declining,0.1971
4,GJ_AMD,2024-05,Stable Urban,0.6211
5,GJ_AMD,2024-06,Declining,0.2238
6,GJ_AMD,2024-07,Declining,0.1638
7,GJ_AMD,2024-08,Stable Urban,0.3374
8,GJ_AMD,2024-09,Stable Urban,0.3021
9,GJ_AMD,2024-10,Declining,0.1422


## Cell 6: Data Quality Validation

In [6]:
"""
Data Quality Checks
-------------------
Validate all required constraints before export.
"""

checks_passed = 0
total_checks = 6

# Check 1: No NaN values
nan_count = df_output.isna().sum().sum()
if nan_count == 0:
    print("✅ Check 1: No NaN values")
    checks_passed += 1
else:
    print(f"❌ Check 1: Found {nan_count} NaN values")

# Check 2: growth_confidence is between 0 and 1
gc_valid = (df_output['growth_confidence'] >= 0).all() and (df_output['growth_confidence'] <= 1).all()
if gc_valid:
    print("✅ Check 2: growth_confidence in range [0, 1]")
    checks_passed += 1
else:
    print(f"❌ Check 2: growth_confidence out of range")

# Check 3: Valid peri_urban_label values
valid_labels = {'Peri-Urban Growth', 'Stable Urban', 'Rural Stable', 'Declining'}
actual_labels = set(df_output['peri_urban_label'].unique())
if actual_labels.issubset(valid_labels):
    print(f"✅ Check 3: Valid label values: {actual_labels}")
    checks_passed += 1
else:
    print(f"❌ Check 3: Invalid labels found: {actual_labels - valid_labels}")

# Check 4: peri_urban_label is non-null
if df_output['peri_urban_label'].notna().all():
    print("✅ Check 4: peri_urban_label is non-null")
    checks_passed += 1
else:
    print("❌ Check 4: peri_urban_label has null values")

# Check 5: Correct column count (exactly 4)
if len(df_output.columns) == 4:
    print("✅ Check 5: Exactly 4 columns (no extra columns)")
    checks_passed += 1
else:
    print(f"❌ Check 5: Expected 4 columns, got {len(df_output.columns)}")

# Check 6: Row count matches input
expected_rows = len(df)
if len(df_output) == expected_rows:
    print(f"✅ Check 6: Row count matches input ({expected_rows} rows)")
    checks_passed += 1
else:
    print(f"❌ Check 6: Row count mismatch. Expected {expected_rows}, got {len(df_output)}")

print(f"\n{'='*50}")
if checks_passed == total_checks:
    print(f"✅ ALL {total_checks} VALIDATION CHECKS PASSED")
else:
    print(f"⚠️ {checks_passed}/{total_checks} checks passed")

✅ Check 1: No NaN values
✅ Check 2: growth_confidence in range [0, 1]
✅ Check 3: Valid label values: {'Peri-Urban Growth', 'Rural Stable', 'Declining', 'Stable Urban'}
✅ Check 4: peri_urban_label is non-null
✅ Check 5: Exactly 4 columns (no extra columns)
✅ Check 6: Row count matches input (240 rows)

✅ ALL 6 VALIDATION CHECKS PASSED


## Cell 7: Export to CSV

In [7]:
"""
Export to CSV
-------------
Save the peri-urban classification data in CSV format.
"""

# Export to CSV (no index column)
df_output.to_csv(OUTPUT_PATH, index=False)

print(f"✅ CSV exported to: {OUTPUT_PATH}")
print(f"\n📊 Export Summary:")
print(f"   Total rows: {len(df_output):,}")
print(f"   Districts: {df_output['district_id'].nunique()}")
print(f"   Months: {df_output['month'].nunique()}")
print(f"\n📊 Label distribution:")
for label, count in df_output['peri_urban_label'].value_counts().sort_index().items():
    pct = count / len(df_output) * 100
    print(f"   {label}: {count} ({pct:.1f}%)")
print(f"\n📋 Schema:")
for col in df_output.columns:
    sample_val = df_output[col].iloc[0]
    print(f"   {col}: {df_output[col].dtype} (e.g., {sample_val})")
print(f"\n📄 First 10 rows:")
df_output.head(10)

✅ CSV exported to: d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\final\peri_urban_labels.csv

📊 Export Summary:
   Total rows: 240
   Districts: 20
   Months: 12

📊 Label distribution:
   Declining: 92 (38.3%)
   Peri-Urban Growth: 1 (0.4%)
   Rural Stable: 90 (37.5%)
   Stable Urban: 57 (23.8%)

📋 Schema:
   district_id: object (e.g., GJ_AMD)
   month: object (e.g., 2024-01)
   peri_urban_label: object (e.g., Stable Urban)
   growth_confidence: float64 (e.g., 0.3711)

📄 First 10 rows:


,district_id,month,peri_urban_label,growth_confidence
0,GJ_AMD,2024-01,Stable Urban,0.3711
1,GJ_AMD,2024-02,Stable Urban,0.4878
2,GJ_AMD,2024-03,Declining,0.2257
3,GJ_AMD,2024-04,Declining,0.1971
4,GJ_AMD,2024-05,Stable Urban,0.6211
5,GJ_AMD,2024-06,Declining,0.2238
6,GJ_AMD,2024-07,Declining,0.1638
7,GJ_AMD,2024-08,Stable Urban,0.3374
8,GJ_AMD,2024-09,Stable Urban,0.3021
9,GJ_AMD,2024-10,Declining,0.1422


## Phase 4 Complete

### Output File
`data/final/peri_urban_labels.csv`

### Schema
| Field | Type | Description |
|-------|------|-------------|
| district_id | string | District identifier |
| month | string | YYYY-MM format |
| peri_urban_label | string | Classification label |
| growth_confidence | float | Confidence score (0-1) |

### Classification Rules Applied
1. **Peri-Urban Growth**: velocity > 0.12 AND youth > 0.38 AND 3m consistent
2. **Declining**: velocity < -0.05
3. **Stable Urban**: youth >= 0.40
4. **Rural Stable**: Default

### Next Phase
Phase 5 will add digital exclusion risk scores.